In [5]:
#ALL IMPORTS
import numpy as np
import pandas as pd

In [6]:
# READING DATA
season17_18 = pd.read_csv("17_18.csv")
season18_19 = pd.read_csv("18_19.csv")
season19_20 = pd.read_csv("19_20.csv")

In [26]:
class Football:
    
    def __init__(self, data, name):
        self.data=data
        self.name=name
    
    def getTeamWins(self):
        #TODO
        return None
    
    def getTeamLoses(self):
        #TODO
        return None
    
    def getTeamDraws(self):
        #TODO
        return None
    
    def getTeamsTotalGoals(self):
        #TODO
        return None
    def Test(self):
        print("Passed")

In [28]:
season1718 = Football(season17_18,"Hooaeg 17-18")
season1718.Test()

Passed
